<a href="https://colab.research.google.com/github/Luxemburgli/BARC-test/blob/master/Arc_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning into 'BARC'...
remote: Enumerating objects: 4395, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 4395 (delta 224), reused 213 (delta 200), pack-reused 4120 (from 2)
Receiving objects: 100% (4395/4395), 17.40 MiB | 34.66 MiB/s, done.
Resolving deltas: 100% (2495/2495), done.


In [ ]:
!pip install vllm
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.1/456.

Here I copy the code from vllm_inference to see if it works

In [ ]:
import time
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest
import datetime
from tqdm import tqdm
import json
import torch
import numpy as np
# Import the modules we need.

In [ ]:
# BASE_MODEL = "mistralai/Codestral-22B-v0.1"
# LORA_DIR = "data/barc-codestral-sft-qlora-v0.0.3-epoch3"
# BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# BASE_MODEL = "./barc-llama3.1-8b-instruct-lora64-induction-gpt4-desc-llama-20k_lr2e-4_epoch3_merged"
# BASE_MODEL = "./barc-llama3.1-8b-instruct-lora64-induction-gpt4-desc-4omini20k_lr2e-4_epoch3_merged"
BASE_MODEL = "barc0/Llama-3.1-ARC-Potpourri-Induction-8B"
# BASE_MODEL = "barc0/barc-llama3.1-8b-instruct-fft-sft-induction35k_lr1e-5_epoch2"
# BASE_MODEL = "data/barc-llama3.1-8b-instruct-fft-induction_gpt4omini100k_lr1e-5_epoch2"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-qlora-v0.0.3"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-qlora-v0.0.2-v3-nopacking"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-lora-data-mix-v0.0.1"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-lora-data-llama-codegen"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-lora-data-gpt4-descriptions"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-lora-data-gpt4omini-codegen"
# LORA_DIR = "data/barc-llama3.1-8b-instruct-sft-lora-gpt-4_description_20000_with_gpt-4o-mini_and_llama3_codegen"
LORA_DIR = None
# LORA_DIR = "barc0/barc-llama3.1-8b-instruct-lora64-induction-gpt4omini35k_lr2e-4_epoch3"

##################################
# The model directory
##################################

# problem_file = "./arc_problems_train_327.jsonl"
# problem_file = "./arc_problems_validation_400.jsonl"
# problem_file = "./arc_problems_selected-val-subset50_50_extra_newline.jsonl"
# problem_file = "./arc_problems_selected-train-subset50_50.jsonl"

# problem_file = "./arc_problems_selected-train-subset50_50_extra_newline.jsonl"
# problem_file = "./arc_problems_train_327_extra_newline.jsonl"
# problem_file = "./arc_problems_validation_400_extra_newline.jsonl"
problem_file = "/content/BARC/finetune/alignment-handbook/arc_problems_validation_400_extra_newline_v2.jsonl"

##################################
# The file directory
##################################

BATCH_SIZE = 16
num_of_samples_per_problem = 64
# There the num_of_samples_per_problem is the total number of samples we want
# Batch size is the number of samples we generate per run, but it could be modified depending on the input size.
TENSOR_PARALLEL = 1
TEMPERATURE = 0.8

if LORA_DIR:
    tokenizer = AutoTokenizer.from_pretrained(LORA_DIR)
else:
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
#################################################
# Load the data

data = []
with open(problem_file) as f:
    for line in f:
        data.append(json.loads(line))

#################################################
# Set the device and load the model


device = "cuda" if torch.cuda.is_available() else "cpu"

if LORA_DIR:
    llm = LLM(model=BASE_MODEL, enable_lora=True, max_lora_rank=256, max_model_len=12000,
            enable_prefix_caching=True, tensor_parallel_size=TENSOR_PARALLEL)
    lora_request=LoRARequest("barc_adapter", 1, LORA_DIR)
else:
    llm = LLM(model=BASE_MODEL, enable_lora=False, max_model_len=12000,
            enable_prefix_caching=True, tensor_parallel_size=TENSOR_PARALLEL, dtype = torch.float16)



#################################################
# Set saving route

datetime_str = datetime.datetime.now().strftime("%m%d%H%M%S%f")
if LORA_DIR:
    saving_file = f"{problem_file.replace('.jsonl', '')}_{LORA_DIR.split('/')[-1]}_temp_{TEMPERATURE}_{datetime_str}.jsonl"
else:
    saving_file = f"{problem_file.replace('.jsonl', '')}_{BASE_MODEL.split('/')[-1]}_temp_{TEMPERATURE}_{datetime_str}.jsonl"
print(f"Saving to {saving_file}")
time.sleep(5)


WARNING 01-22 22:36:11 config.py:2276] Casting torch.bfloat16 to torch.float16.
INFO 01-22 22:36:11 config.py:510] This model supports multiple tasks: {'embed', 'generate', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
INFO 01-22 22:36:11 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='barc0/Llama-3.1-ARC-Potpourri-Induction-8B', speculative_config=None, tokenizer='barc0/Llama-3.1-ARC-Potpourri-Induction-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=12000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_trac

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 635.06 MiB is free. Process 13792 has 14.12 GiB memory in use. Of the allocated memory 14.01 GiB is allocated by PyTorch, and 1.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
all_responses = []
for d in tqdm(data):
    messages = d["messages"]
    assert messages[0]["role"] == "system"
    assert messages[1]["role"] == "user"
    inputs = tokenizer.apply_chat_template([
        {"role":"system", "content":messages[0]["content"]},
        {"role":"user", "content":messages[1]["content"]}
    ], tokenize=False, add_generation_prompt=True)
    input_tokens = tokenizer.apply_chat_template([
        {"role":"system", "content":messages[0]["content"]},
        {"role":"user", "content":messages[1]["content"]}
    ], tokenize=True, add_generation_prompt=True)
    # print(inputs)
    print(f"Number of tokens: {len(input_tokens)}")
    if len(input_tokens) > 8000:
        print("skip!!!!!")
        continue

    assert num_of_samples_per_problem % BATCH_SIZE == 0
    if  len(input_tokens) < 1750:
        tmp_batch_size = BATCH_SIZE * 4
    elif len(input_tokens) < 4000:
        # double the number of samples
        tmp_batch_size = BATCH_SIZE * 4
    elif len(input_tokens) < 5000:
        tmp_batch_size = BATCH_SIZE
    else:
        tmp_batch_size = BATCH_SIZE

    print(f"batch size: {tmp_batch_size}")
    sampling_params = SamplingParams(temperature=TEMPERATURE, max_tokens=1536,
                                     n=tmp_batch_size)
    aggregate_outputs = []
    for i in range(num_of_samples_per_problem // tmp_batch_size):
        if LORA_DIR:
            outputs = llm.generate(
                inputs,
                sampling_params,
                lora_request=lora_request
            )
        else:
            outputs = llm.generate(
                inputs,
                sampling_params,
            )
        aggregate_outputs.append(outputs)

    if not aggregate_outputs:
        breakpoint()
    else:
        print(aggregate_outputs[0])


    # Print the outputs.
    responses = []
    for outputs in aggregate_outputs:
        for output in outputs:
            prompt = output.prompt
            print(f"Prompt: {prompt!r}")
            for i in range(len(output.outputs)):
                generated_text = output.outputs[i].text
                # print(f"Generated text: {generated_text!r}\n")
                responses.append(generated_text)

    all_responses.append({"uid": d["uid"], "prompt":inputs , "responses": responses, "base_model": BASE_MODEL, "lora_dir": LORA_DIR})

    with open(saving_file, "w") as f:
        f.write("\n".join(json.dumps(p) for p in all_responses))

print(f"Saving to {saving_file}")

time.sleep(15)
